In [1]:
import folium
import os, shutil
from glob import glob
#%matplotlib inline  
import json
import satsearch
import geopandas as gpd
import numpy as np
import pandas as pd
import warnings
from shapely.geometry import  Polygon
import rioxarray
import satstac



In [2]:
from src import cropy as cpy


In [3]:
import sys
print(sys.path)

['', 'C:\\Users\\akif\\Documents\\geospatial_process\\notebooks', 'C:\\Users\\akif\\.conda\\envs\\geo\\python37.zip', 'C:\\Users\\akif\\.conda\\envs\\geo\\DLLs', 'C:\\Users\\akif\\.conda\\envs\\geo\\lib', 'C:\\Users\\akif\\.conda\\envs\\geo', 'C:\\Users\\akif\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\Users\\akif\\.conda\\envs\\geo\\lib\\site-packages', 'C:\\Users\\akif\\.conda\\envs\\geo\\lib\\site-packages\\win32', 'C:\\Users\\akif\\.conda\\envs\\geo\\lib\\site-packages\\win32\\lib', 'C:\\Users\\akif\\.conda\\envs\\geo\\lib\\site-packages\\Pythonwin', 'C:\\Users\\akif\\.conda\\envs\\geo\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\akif\\.ipython']


In [4]:
import sys
sys.path.insert(0, "C:/Users/Documents/geospatial_process/src")

In [5]:
sys.path.append(r"C:\Users\Documents\geospatial_process\src\data")

In [6]:
harran={
"type": "FeatureCollection",
"name": "Harran_AOI",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "fid": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 38.680367610550277, 36.71081864055018 ], [ 38.688923838082594, 37.274751818816462 ], [ 39.282414893278698, 37.277085335416182 ], [ 39.29019328194444, 36.690594830019251 ], [ 38.680367610550277, 36.71081864055018 ] ] ] } }
]
}


In [7]:
datajson=json.dumps(harran)
area=gpd.read_file(datajson)


In [8]:
cpy.VectorProcessing.show_vector(target_area=area)

In [9]:
m,intersec_df=cpy.VectorProcessing.show_intersection(target_area=area,base_vector_path='../input_data/raw/boundries/sentinel_tr_tiles.shp')

In [10]:
m

In [11]:
intersec_df.head()

,name,folders,descriptio,altitude,alt_mode,time_begin,time_end,time_when,GID_0,NAME_0,geometry
0,37SDA,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((37.87506 37.04125 0.00000, 39.1097..."
1,37SDB,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((37.86147 37.94208 0.00000, 39.1110..."
2,37SEA,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((38.99978 37.04658 0.00000, 40.2344..."
3,37SEB,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((38.99977 37.94759 0.00000, 40.2493..."


In [30]:
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2018-08-01/2018-12-01'
band_list=['B02','B08']
cloud_percentage=30

In [31]:
#burasu stact query olmali
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)

In [32]:
#get tile list of stac query result
stac_query.create_tiles_list()

['37SDA', '37SDB', '37SEA', '37SEB']

In [15]:
#get only target tiles. change the tiles_list of query
# default, query get all tiles that intersect with user AOI
#stac_query.tiles_list=['37SEA']

In [33]:
sentinel_items=stac_query.find_sentinel_item()

In [34]:
sentinel_items

[S2A_37SDA_20181030_0_L2A,
 S2B_37SDB_20180925_0_L2A,
 S2A_37SEA_20181030_0_L2A,
 S2A_37SEB_20181030_0_L2A]

In [35]:
items_list=stac_query.stac_items

For show_result_df and show_result_list methods, you can use stac_query.find_sentinel_item() which means best image for your target area and dates. You can read deatails with stac_query.find_sentinel_item? command<br/>

If you see all available

In [38]:
'''
You can use this method to get image with pandas queries. After define your target image, you can use 'sentinel:product_id' information with stac_query.stac_result() in downloand methods.
'''
df=stac_query.show_result_df(items_list=items_list)
df.head()

,datetime,platform,constellation,instruments,gsd,data_coverage,view:off_nadir,eo:cloud_cover,proj:epsg,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,created,updated,sentinel:valid_cloud_cover,sentinel:utm_zone,sentinel:data_coverage,geometry
0,2018-10-30 08:12:29,sentinel-2a,sentinel-2,msi,10,100.0,0,0,32637,S,DA,0,S2A_MSIL2A_20181030T081041_N0209_R078_T37SDA_2...,2020-09-26T02:48:18.340Z,2020-09-26T02:48:18.340Z,True,37,100,"POLYGON ((37.88933 36.05159, 37.87508 37.04124..."
1,2018-09-25 08:11:52,sentinel-2b,sentinel-2,msi,10,100.0,0,0,32637,S,DB,0,S2B_MSIL2A_20180925T080649_N0208_R078_T37SDB_2...,2020-10-02T11:33:29.791Z,2020-10-02T11:33:29.791Z,True,37,100,"POLYGON ((37.87638 36.95257, 37.86148 37.94207..."
2,2018-10-30 08:12:29,sentinel-2a,sentinel-2,msi,10,NaN,0,0,32637,S,EA,0,S2A_MSIL2A_20181030T081041_N0209_R078_T37SEA_2...,2020-08-31T01:47:12.102Z,2020-08-31T01:47:12.102Z,True,37,100,"POLYGON ((40.21879 36.05054, 38.99979 36.05673..."
3,2018-10-30 08:12:29,sentinel-2a,sentinel-2,msi,10,100.0,0,0,32637,S,EB,0,S2A_MSIL2A_20181030T081041_N0209_R078_T37SEB_2...,2020-09-08T13:09:09.775Z,2020-09-08T13:09:09.775Z,True,37,100,"POLYGON ((40.23299 36.95149, 38.99979 36.95789..."


In [41]:
'''
Use for direct integration into Qgis or We can use it for internal API services.
'''
result_list=stac_query.show_result_list(sentinel_items_list=items_list)

In [42]:
result_list

[{'tile_name': '37SDA',
  'tile_images': {'image_name': 'S2A_MSIL2A_20181109T081141_N0210_R078_T37SDA_20181109T104132',
   'bands': {'B01': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/11/S2A_37SDA_20181109_0_L2A/B01.tif',
    'B02': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/11/S2A_37SDA_20181109_0_L2A/B02.tif',
    'B03': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/11/S2A_37SDA_20181109_0_L2A/B03.tif',
    'B04': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/11/S2A_37SDA_20181109_0_L2A/B04.tif',
    'B05': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/11/S2A_37SDA_20181109_0_L2A/B05.tif',
    'B06': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/11/S2A_37SDA_20181109_0_L2A/B06.tif',
    'B07': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-

In [27]:
stac_query.show_result_map(items_list=sentinel_items,
                           target_area=area,
                           overview=True)

In [ ]:
stac_query.download_image(item_list=sentinel_items[1:2],band_list=band_list[0:1])

In [121]:
stac_query.download_image(item_list=sentinel_items[0:1],
                                              band_list=band_list[0:1],
                                              #download_status=True,
                                              #download_path='./target_path'
                                              )

'./sentinel_cog'

In [43]:
# get data as xarray
subset_image=stac_query.download_subset_image(item_list=sentinel_items[0:1],
                                              band_list=band_list[0:1],
                                              aoi=harran,
                                              download_status=True,
                                              #download_path='./target_path'
                                              )

### After getting error from COG images

You can use download_error_image(img_date,geo_img,img_id,username,password) method in stac.query class. 
Write stac_query.download_error_image? command and read the description.

Below code, you can see download_error_image method's details

In [171]:
stac_items=stac_query.stac_items

In [159]:
img_id=stac_items[0].properties["sentinel:product_id"]
img_id

'S2B_MSIL2A_20181124T081249_N0211_R078_T37SEA_20181124T095227'

In [136]:
img_geo=stac_items[0].geometry

In [124]:
import requests

In [125]:
url='https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B01.tif'
incorrect_url='https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/EB/2020/7/S2A_37SEB_20200731_0_L2A/B02.tif'

In [126]:
resp = requests.get(f'http://cog-validate.radiant.earth/api/validate?url={url}')
resp.json()['status']

'success'

In [130]:
resp = requests.get(f'http://cog-validate.radiant.earth/api/validate?url={incorrect_url}')
resp.json()['status']

'failure'

In [131]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import datetime

In [138]:
api = SentinelAPI('orttak', 'a2k5i4f313', 'https://scihub.copernicus.eu/dhus')

In [139]:
date = datetime.date(2020, 2, 20)
date += datetime.timedelta(days=1)
date

datetime.date(2020, 2, 21)

In [145]:
tt =stac_items[0].date- datetime.timedelta(days=1)
tt

datetime.date(2018, 11, 23)

In [148]:
fr=stac_items[0].date+datetime.timedelta(days=1)
fr

datetime.date(2018, 11, 25)

In [151]:
# search by polygon, time, and Hub query keywords
footprint = geojson_to_wkt(img_geo)
products = api.query(footprint,
                     #date = ('20181219', date(2018, 12, 29)),
                     date=(tt,fr),
                     platformname = 'Sentinel-2',
                     cloudcoverpercentage = (0, 90))

In [155]:
sat_df=api.to_geodataframe(products)
sat_df.head(1)

,title,link,link_alternative,link_icon,summary,ondemand,ingestiondate,beginposition,endposition,orbitnumber,...,size,uuid,datatakesensingstart,sensoroperationalmode,tileid,hv_order_tileid,level1cpdiidentifier,granuleidentifier,datastripidentifier,geometry
0d601818-ce7d-4557-a61a-2ba6ccddc6d9,S2B_MSIL2A_20181124T081249_N0211_R078_T37SEA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-11-24T08:12:49.024Z, Instrument: MS...",false,2018-11-24 13:57:41.907,2018-11-24 08:12:49.024,2018-11-24 08:12:49.024,8970,...,996.72 MB,0d601818-ce7d-4557-a61a-2ba6ccddc6d9,NaT,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((38.99978 37.04658, 40.23444 37.04016..."


In [153]:
res=sat_df.loc[sat_df['title']==img_id]

In [154]:
res.index.values[0]

'0d601818-ce7d-4557-a61a-2ba6ccddc6d9'

In [ ]:
api.download_all(res.index,directory_path='.')

In [ ]:
api.download('7be30c50-31fc-48c4-ab45-fddea9be7877',directory_path='.')

In [ ]:
api.get_product_odata('c59cdd69-3af0-48cb-9e7f-9bfde928d3cd')

In [ ]:
api.download('37b60075-6921-4e20-ba20-fbc1c8393e66',directory_path='.')

In [ ]:
api.get_product_odata('S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148')

In [ ]:
sat_df

In [ ]:
sat_df.iloc[12] .title

In [ ]:
geo1.to_wkt()

In [ ]:
sat_df.iloc[0].filename

In [ ]:
a='S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148'
b='S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148'
a==b